In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys

# making sure the dl4mt module is on the path 
# -- this path depends upon the location where the notebook is running
# here it is assumed to be the day1/ directory, change it to point to the location of the dl4mt/ dir if necessary 
sys.path.append('../..')

In [11]:
%%writefile ../../dl4mt/mlp.py
# Uncomment to save this cell to file in order to import it in later notebooks

# construct an MLP graph using theano, then train and evaluate using static features
import numpy
import theano
import theano.tensor as T

# this is our logistic regression class, created and saved in the LogisticRegression notebook
from dl4mt.logistic_regression import LogisticRegression

class HiddenLayer(object):
    def __init__(self, rng, input, n_in, n_out, W=None, b=None,
                 activation=T.tanh):
        """
        Typical hidden layer of a MLP: units are fully-connected and have
        sigmoidal activation function. Weight matrix W is of shape (n_in,n_out)
        and the bias vector b is of shape (n_out,).

        Hidden unit activation is given by: tanh(dot(input,W) + b)

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.dmatrix
        :param input: a symbolic tensor of shape (n_examples, n_in)

        :type n_in: int
        :param n_in: dimensionality of input

        :type n_out: int
        :param n_out: number of hidden units

        :type activation: theano.Op or function
        :param activation: Non linearity to be applied in the hidden
                           layer
        """
        self.input = input

        # `W` is initialized with `W_values` which is uniformely sampled
        # from sqrt(-6./(n_in+n_hidden)) and sqrt(6./(n_in+n_hidden))
        # for tanh activation function
        # the output of uniform if converted using asarray to dtype
        # theano.config.floatX so that the code is runable on GPU
        # Note : optimal initialization of weights is dependent on the
        #        activation function used (among other things).
        #        For example, results presented in [Xavier10] suggest that you
        #        should use 4 times larger initial weights for sigmoid
        #        compared to tanh
        #        We have no info for other function, so we use the same as
        #        tanh.
        if W is None:
            W_values = numpy.asarray(
                rng.uniform(
                    low=-numpy.sqrt(6. / (n_in + n_out)),
                    high=numpy.sqrt(6. / (n_in + n_out)),
                    size=(n_in, n_out)
                ),
                dtype=theano.config.floatX
            )
            if activation == theano.tensor.nnet.sigmoid:
                W_values *= 4

            W = theano.shared(value=W_values, name='W', borrow=True)

        if b is None:
            b_values = numpy.zeros((n_out,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b', borrow=True)

        self.W = W
        self.b = b

        lin_output = T.dot(input, self.W) + self.b
        self.output = (
            lin_output if activation is None
            else activation(lin_output)
        )
        # parameters of the model
        self.params = [self.W, self.b]


class MLP(object):
    """Multi-Layer Perceptron Class

    A multilayer perceptron is a feedforward artificial neural network model
    that has one layer or more of hidden units and nonlinear activations.
    Intermediate layers usually have as activation function tanh or the
    sigmoid function (defined here by a ``HiddenLayer`` class)  while the
    top layer is a softmax layer (defined here by a ``LogisticRegression``
    class).
    """

    def __init__(self, rng, input, n_in, n_hidden, n_out):
        """Initialize the parameters for the multilayer perceptron

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.TensorType
        :param input: symbolic variable that describes the input of the
        architecture (one minibatch)

        :type n_in: int
        :param n_in: number of input units, the dimension of the space in
        which the datapoints lie

        :type n_hidden: int
        :param n_hidden: number of hidden units

        :type n_out: int
        :param n_out: number of output units, the dimension of the space in
        which the labels lie

        """

        # Since we are dealing with a one hidden layer MLP, this will translate
        # into a HiddenLayer with a tanh activation function connected to the
        # LogisticRegression layer; the activation function can be replaced by
        # sigmoid or any other nonlinear function
        self.hiddenLayer = HiddenLayer(
            rng=rng,
            input=input,
            n_in=n_in,
            n_out=n_hidden,
            activation=T.tanh
        )

        # The logistic regression layer gets as input the hidden units
        # of the hidden layer
        self.logRegressionLayer = LogisticRegression(
            input=self.hiddenLayer.output,
            n_in=n_hidden,
            n_out=n_out
        )
        
        # alias to logRegressionLayer.y_pred to provide a consistent interface to our models
        self.y_pred = self.logRegressionLayer.y_pred

        # L1 norm; one regularization option is to enforce L1 norm to be small
        self.L1 = (
            abs(self.hiddenLayer.W).sum()
            + abs(self.logRegressionLayer.W).sum()
        )

        # square of L2 norm ; one regularization option is to enforce
        # square of L2 norm to be small
        self.L2_sqr = (
            (self.hiddenLayer.W ** 2).sum()
            + (self.logRegressionLayer.W ** 2).sum()
        )

        # negative log likelihood of the MLP is given by the negative
        # log likelihood of the output of the model, computed in the
        # logistic regression layer
        self.negative_log_likelihood = (
            self.logRegressionLayer.negative_log_likelihood
        )
        # same holds for the function computing the number of errors
        self.errors = self.logRegressionLayer.errors

        # the parameters of the model are the parameters of the two layer it is
        # made out of
        self.params = self.hiddenLayer.params + self.logRegressionLayer.params

        # keep track of model input -- this will be useful for creating theano functions
        self.input = input

Overwriting ../../dl4mt/mlp.py


In [12]:
import numpy

import theano
import theano.tensor as T
import timeit

from dl4mt.mlp import MLP

# WORKING: change the signature of this function to return

def train_mlp(train_dataset, dev_dataset, learning_rate=0.01, L1_reg=0.00, L2_reg=0.0001,
              training_epochs=25, batch_size=10, n_hidden=500):
    
    """
    Demonstrate stochastic gradient descent optimization for a multilayer
    perceptron

    :type learning_rate: float
    :param learning_rate: learning rate used (factor for the stochastic
    gradient

    :type L1_reg: float
    :param L1_reg: L1-norm's weight when added to the cost (see
    regularization)

    :type L2_reg: float
    :param L2_reg: L2-norm's weight when added to the cost (see
    regularization)

    :type training_epochs: int
    :param training_epochs: maximal number of epochs to run the optimizer

    :type dataset: string
    :param dataset: the path of the MNIST dataset file from
                 http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz


   """

    train_set_x, train_set_y = train_dataset
    valid_set_x, valid_set_y = dev_dataset

    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
#     n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size
    
    # TODO: this part of the training is common to all of the models
    ######################
    # BUILD ACTUAL MODEL #
    ######################
    print '... building the model'

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch
    x = T.matrix('x')  # the data is presented as rasterized images
    y = T.ivector('y')  # the labels are presented as 1D vector of
                        # [int] labels

    rng = numpy.random.RandomState(1234)
    
    # TODO: move these into the parameterization of the training function
    n_in = train_set_x.get_value().shape[1]
    n_out = 12
    
    # construct the MLP class
    classifier = MLP(
        rng=rng,
        input=x,
        n_in=n_in,
        n_hidden=n_hidden,
        n_out=n_out
    )

    # the cost we minimize during training is the negative log likelihood of
    # the model plus the regularization terms (L1 and L2); cost is expressed
    # here symbolically
    cost = (
        classifier.negative_log_likelihood(y)
        + L1_reg * classifier.L1
        + L2_reg * classifier.L2_sqr
    )

    # compiling a Theano function that computes the mistakes that are made
    # by the model on a minibatch
#     test_model = theano.function(
#         inputs=[index],
#         outputs=classifier.errors(y),
#         givens={
#             x: test_set_x[index * batch_size:(index + 1) * batch_size],
#             y: test_set_y[index * batch_size:(index + 1) * batch_size]
#         }
#     )

    validate_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size:(index + 1) * batch_size],
            y: valid_set_y[index * batch_size:(index + 1) * batch_size]
        }
    )

    # compute the gradient of cost with respect to theta (sotred in params)
    # the resulting gradients will be stored in a list gparams
    gparams = [T.grad(cost, param) for param in classifier.params]

    # specify how to update the parameters of the model as a list of
    # (variable, update expression) pairs

    # given two lists of the same length, A = [a1, a2, a3, a4] and
    # B = [b1, b2, b3, b4], zip generates a list C of same size, where each
    # element is a pair formed from the two lists :
    #    C = [(a1, b1), (a2, b2), (a3, b3), (a4, b4)]
    updates = [
        (param, param - learning_rate * gparam)
        for param, gparam in zip(classifier.params, gparams)
    ]

    # compiling a Theano function `train_model` that returns the cost, but
    # in the same time updates the parameters of the model based on the rules
    # defined in `updates`
    train_model = theano.function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    ###############
    # TRAIN MODEL #
    ###############
    print '... training'

    # early-stopping parameters
    patience = 10000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                           # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                   # considered significant
    validation_frequency = min(n_train_batches, patience / 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = numpy.inf
    best_iter = 0
    test_score = 0.
    start_time = timeit.default_timer()

    epoch = 0
    done_looping = False

    while (epoch < training_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in xrange(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i) for i
                                     in xrange(n_valid_batches)]
                this_validation_loss = numpy.mean(validation_losses)

                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_validation_loss * 100.
                    )
                )

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if (
                        this_validation_loss < best_validation_loss *
                        improvement_threshold
                    ):
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    best_iter = iter

            if patience <= iter:
                done_looping = True
                break

    end_time = timeit.default_timer()
    print(('Optimization complete. Best validation score of %f %% '
           'obtained at iteration %i') %
          (best_validation_loss * 100., best_iter + 1))
    return classifier

In [16]:
# TODO: allow user to load another distributed representation for comparison, such as PCA
import os
from dl4mt.datasets import prep_dataset

# load the training data
DATASET_LOCATION = 'datasets/'

# the pos dataset consists of windows around words
POS_DATASET_NAME = 'brown_pos_dataset.hdf5'
POS_DATASET_PATH = os.path.join(DATASET_LOCATION, POS_DATASET_NAME)
WORD_BY_WORD_MATRIX = 'brown.word-by-word.normalized.npy'
VECTOR_INDEX_PATH = os.path.join(DATASET_LOCATION, WORD_BY_WORD_MATRIX)

CUTOFF = 100000
  
train_dataset = prep_dataset(POS_DATASET_PATH, VECTOR_INDEX_PATH, which_sets=['train'], cutoff=CUTOFF)
dev_dataset = prep_dataset(POS_DATASET_PATH, VECTOR_INDEX_PATH, which_sets=['dev'], cutoff=CUTOFF)

# train the MLP
# L2_reg=0.0001
mlp_model = train_mlp(train_dataset, dev_dataset, learning_rate=0.01, L1_reg=0.00, L2_reg=0.001,
                                     training_epochs=100, batch_size=20, n_hidden=500)

... building the model
... training
epoch 1, minibatch 5000/5000, validation error 17.794720 %
epoch 2, minibatch 5000/5000, validation error 15.987711 %
epoch 3, minibatch 5000/5000, validation error 15.300410 %
epoch 4, minibatch 5000/5000, validation error 14.959035 %
epoch 5, minibatch 5000/5000, validation error 14.776969 %
epoch 6, minibatch 5000/5000, validation error 14.651798 %
epoch 7, minibatch 5000/5000, validation error 14.544834 %
epoch 8, minibatch 5000/5000, validation error 14.497041 %
epoch 9, minibatch 5000/5000, validation error 14.444697 %
epoch 10, minibatch 5000/5000, validation error 14.408284 %
epoch 11, minibatch 5000/5000, validation error 14.362767 %
epoch 12, minibatch 5000/5000, validation error 14.299044 %
epoch 13, minibatch 5000/5000, validation error 14.278562 %
Optimization complete. Best validation score of 14.278562 % obtained at iteration 65000


In [19]:
from dl4mt.predict import predict

test_dataset = prep_dataset(POS_DATASET_PATH, VECTOR_INDEX_PATH, which_sets=['test'], cutoff=CUTOFF, cast_y=False)

test_X, test_y = test_dataset
predictions = predict(mlp_model, test_X.get_value())
predictions
zip(predictions, list(test_y.get_value()))

[(1, 1.0),
 (1, 1.0),
 (2, 2.0),
 (1, 1.0),
 (1, 7.0),
 (6, 6.0),
 (1, 7.0),
 (2, 2.0),
 (4, 4.0),
 (1, 1.0),
 (2, 2.0),
 (1, 7.0),
 (1, 11.0),
 (1, 1.0),
 (5, 5.0),
 (3, 3.0),
 (7, 7.0),
 (4, 4.0),
 (1, 1.0),
 (11, 11.0),
 (1, 11.0),
 (1, 1.0),
 (6, 6.0),
 (7, 7.0),
 (4, 4.0),
 (5, 5.0),
 (11, 11.0),
 (1, 1.0),
 (5, 5.0),
 (5, 5.0),
 (10, 10.0),
 (1, 1.0),
 (1, 1.0),
 (1, 1.0),
 (1, 1.0),
 (1, 1.0),
 (2, 2.0),
 (1, 1.0),
 (7, 7.0),
 (4, 0.0),
 (5, 5.0),
 (10, 10.0),
 (3, 3.0),
 (7, 7.0),
 (7, 7.0),
 (4, 4.0),
 (1, 1.0),
 (5, 5.0),
 (5, 5.0),
 (3, 3.0),
 (7, 7.0),
 (7, 0.0),
 (7, 7.0),
 (0, 0.0),
 (11, 11.0),
 (5, 5.0),
 (5, 5.0),
 (3, 3.0),
 (7, 7.0),
 (5, 5.0),
 (3, 3.0),
 (7, 7.0),
 (7, 7.0),
 (2, 2.0),
 (1, 1.0),
 (1, 0.0),
 (3, 3.0),
 (7, 7.0),
 (7, 7.0),
 (6, 6.0),
 (7, 7.0),
 (11, 11.0),
 (1, 1.0),
 (2, 2.0),
 (11, 11.0),
 (1, 1.0),
 (5, 5.0),
 (1, 0.0),
 (5, 5.0),
 (1, 6.0),
 (7, 7.0),
 (11, 11.0),
 (1, 1.0),
 (5, 5.0),
 (1, 1.0),
 (7, 7.0),
 (4, 4.0),
 (11, 7.0),
 (1, 1.0),
 (

In [20]:
# Quick Evaluation

sum([y==p for y,p in zip(predictions, list(test_y.get_value()))]) / float(len(predictions))

0.8640851799608682

In [18]:
# Challenge: change the MLP class to allow adding multiple hidden layers 
# Challenge: compare performance with different kinds of regularization


In [ ]:
"""
This tutorial introduces the multilayer perceptron using Theano.

 A multilayer perceptron is a logistic regressor where
instead of feeding the input to the logistic regression you insert a
intermediate layer, called the hidden layer, that has a nonlinear
activation function (usually tanh or sigmoid) . One can use many such
hidden layers making the architecture deep. The tutorial will also tackle
the problem of MNIST digit classification.

.. math::

    f(x) = G( b^{(2)} + W^{(2)}( s( b^{(1)} + W^{(1)} x))),

References:

    - textbooks: "Pattern Recognition and Machine Learning" -
                 Christopher M. Bishop, section 5

"""
__docformat__ = 'restructedtext en'